# FastText

## 1. 原理
包含三部分，模型架构，层次$SoftMax$和$N-gram$子词特征

$FastText$ = $word2vec$中$cbow$ + $h-softmax$的灵活使用

### 1.1 $CBOW$模型
输入的是$w(t)$的上下文$2d$个词，经过隐藏层后，输出的是$w(t)$  

![CBOW架构](https://note.youdao.com/yws/api/personal/file/D96181D8E0B5464495715A60EE33A4A4?method=download&shareKey=d366693a06d77aecd2c8c8e53786c8fc)

### 1.2 $FastText$模型
其中$x_1,x_2,...,x_{N−1},x_N$表示一个文本中的$n-gram$向量，每个特征是词向量的平均值。这和前文中提到的$CBOW$相似，$CBOW$用上下文去预测中心词，而此处用全部的$n-gram$去预测指定类别。  

![](https://note.youdao.com/yws/api/personal/file/A5695F25596D4311835B2DA25AE8269B?method=download&shareKey=f97031192717a3d0c76c80f649a4344e)

### $层次SoftMax(Hierarchical softmax)$
1. 对于有大量类别的数据集，$fastTex$使用了一个分层分类器（而非扁平式架构）。不同的类别被整合进树形结构中。在某些文本分类任务中类别很多，计算线性分类器的复杂度高。为了改善运行时间，$fastText$ 模型使用了层次 $Softmax$ 技巧。层次 $Softmax$ 技巧建立在哈弗曼编码的基础上，对标签进行编码，能够极大地缩小模型预测目标的数量。

2. fastText 也利用了类别（class）不均衡这个事实（一些类别出现次数比其他的更多），通过使用 $Huffman$ 算法建立用于表征类别的树形结构。因此，频繁出现类别的树形结构的深度要比不频繁出现类别的树形结构的深度要小，这也使得进一步的计算效率更高。

![](https://note.youdao.com/yws/api/personal/file/B5799630F82E41B890E7ED1CFAFA5ECE?method=download&shareKey=253bbd75f5d1fea32b6637f6d37877f3)

### 1.3 $n-gram$子词特征
$fasttext$ 可以用于文本分类和句子分类。不管是文本分类还是句子分类，我们常用的特征是词袋模型。但词袋模型不能考虑词之间的顺序，因此 $fasttext$ 还加入了 $n-gram$ 特征。在 $fasttext$ 中，每个词被看做是$n-gram$字母串包。为了区分前后缀情况，$"<", ">"$符号被加到了词的前后端。除了词的子串外，词本身也被包含进了$n-gram$字母串包。

如果$n=3$，也叫作$trigram$,最小切分单位为字，则“欢迎关注数据科学杂谈”这个句子的$3-gram$为{”欢迎关”, “迎关注”, "关注数”, “注数据”, “数据科”, "据科学”, “科学杂”, ”学杂谈”}。

例如$"where\ are\ you"$使用3-gram后:$”<wh”、 "whe"、 "her”、 "ere”、 "re>"、 ”< ar”、“are"、"re>""<yo"、 "you”、 "ou>"$  
在英文中使用字词$N-gram$可以获取细致的特征，例如$"ed>"$ 可能代表过去时，$"ly>"$可能代表副词等等。


## 2. $keras$实现$FastText$

```python
# coding: utf-8
from __future__ import unicode_literals

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense

VOCAB_SIZE = 2000
EMBEDDING_DIM = 100
MAX_WORDS = 500
CLASS_NUM = 5

def build_fastText():
    model = Sequential()
    
    # 通过embedding层，我们将词汇映射成EMBEDDING_DIM维向量。
    model.add(Embeddding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_WORDS))
    
    # 通过GlobalAveragePooling1D，我们平均了文档中所有词的embedding
    model.add(GlobalAveragePooling1D())
    
    # 通过输出层Softmax分类（真实的fastText这里分层Softmax），得到类别概率分布
    model.add(Dense(CLASS_NUM, activation="softmax"))
    
    # 定义损失函数、优化器、分类度量指标
    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    
    return model

if __name__ == '__main__':
    model = build_fastText()
    print(model.summary())
```

## 3. 实践

In [2]:
import pandas as pd
from sklearn.metrics import f1_score

# 转化为FastText需要的格式
train_df = pd.read_csv("input/train_set.csv", sep='\t')
train_df["label_ft"] = '__label__' + train_df['label'].astype(str)

In [5]:
train_df["label_ft"].head()

0     __label__2
1    __label__11
2     __label__3
3     __label__2
4     __label__3
Name: label_ft, dtype: object

In [6]:
train_df[['text','label_ft']].iloc[:-5000].to_csv("train.csv", index=None, header=None, sep='\t')

In [8]:
train = pd.read_csv("train.csv", sep='\t')

In [12]:
train.head()

2967 6758 339 2021 1854 3731 4109 3792 4149 1519 2058 3912 2465 2410 1219 6654 7539 264 2456 4811 1292 2109 6905 5520 7058 6045 3634 6591 3530 6508 2465 7044 1519 3659 2073 3750 3731 4109 3792 6831 2614 3370 4269 3370 486 5770 4109 4125 3750 5445 2466 6831 6758 3743 3630 1726 2313 5906 826 4516 657 900 1871 7044 3750 2967 3731 1757 1939 648 2828 4704 7039 3706 3750 965 2490 7399 3743 2145 2407 7451 3775 6017 5998 1641 299 4704 2621 7029 3056 6333 433 648 1667 1099 900 2289 1099 648 5780 220 7044 1279 7426 4269 3750 2967 6758 6631 3099 2205 7305 2620 5977 3750 3329 1793 6666 2042 3193 4149 1519 7039 3706 2446 5399 648 4124 2058 3912 248 3193 2252 5649 2212 4939 7239 3310 4525 2400 900 5770 4109 4125 7044 4921 265 1397 4699 1699 669 6407 3750 1271 1271 4741 669 4659 3870 4030 4167 5338 25 3466 6909 4417 1859 3750 1465 7194 648 3938 1571 848 6986 827 2124 3750 1991 7444 7037 2729 908 6308 3750 1889 6810 4190 591 5598 2289 2109 6831 6407 2400 5410 517 900 25 3731 4109 3792 4128 1679 4811 4853 4109 3630 6902 6122 1903 1736 3915 2602 6822 3750 6630 4265 591 729 4448 648 1465 1401 4853 648 5881 6182 4128 1679 4939 2646 652 340 7328 1320 900 1460 619 5505 2376 4853 3272 3750 4853 4109 3630 6902 3362 2810 3750 803 1985 4128 669 19 6508 900 1635 1871 7377 6122 6017 3750 2289 1099 3938 1571 7509 1375 5393 5589 5037 2115 4707 5310 6811 6093 900 7399 2410 1219 6654 3263 6017 3750 5998 4939 5971 4148 3750 803 1985 7194 4780 796 6038 4231 648 1722 6407 3750 1099 6485 1920 1767 5915 6518 6093 5598 5648 4280 900 7326 6242 5328 1214 3870 1985 7194 5998 5741 2115 913 5950 3800 1538 686 6734 6017 3750 1985 3659 1324 5814 4998 5176 535 7399 307 4068 486 1667 1099 2121 6407 3750 7420 3099 6038 4231 4190 1519 3255 7123 4305 3231 1635 4822 1722 3750 2967 3731 1757 1939 648 473 6518 2400 2614 5330 5530 1394 4939 1903 7495 7239 900 4469 5530 4704 299 7467 2121 669 5693 3750 3618 299 5264 4853 1734 316 2828 5445 4190 4939 3484 6043 2376 1031 761 900 5370 3782 2210 669 2210 3099 1363 6301 3508 1907 2410 7509 5718 541 3750 803 2967 6758 3038 6641 1985 7194 512 4811 6811 5243 2112 3750 1734 2376 2891 1211 648 7257 4148 7159 1667 3750 5816 4202 2400 5864 3915 7399 3414 1667 5977 7327 7256 2935 4936 1667 2151 900 6831 4599 6182 3227 3859 3099 7509 7256 3750 1985 7194 4128 4691 2029 1344 6630 5598 1465 648 3706 7403 543 3038 900 1985 7194 3800 980 6017 980 4124 648 900 1635 3605 5028 3731 4109 3792 1866 3578 3915 648 4939 1335 6666 6560 3750 3618 3508 1907 2410 1913 6656 3750 2828 4704 4998 4939 7039 3915 4167 5338 3750 803 1985 4939 3263 7123 264 2456 5689 2109 648 3750 6093 1699 5589 4411 1866 4750 648 1667 1099 3000 7420 1279 2975 1141 7148 3750 1985 3915 2570 4936 5998 1877 3000 7420 900 1635 5470 2313 5864 641 4333 3750 3915 5659 316 2828 2770 5176 803 2047 7532 606 6980 1635 3750 803 1750 7039 3800 7245 3099 7509 5839 3750 1866 1401 4321 5788 1519 6122 6405 4939 5998 2729 900 1985 7194 5998 2289 2107 1519 1592 316 2828 1679 4811 5461 3324 4525 4052 3750 2212 742 3750 1985 7194 6631 1335 5445 3750 1465 7194 4128 6887 4819 5977 3223 2717 900 5612 5948 3750 1985 7194 2289 913 3800 4811 6122 2614 2047 7532 606 6980 900 1985 2541 4409 3772 6012 1833 5560 4173 6662 414 340 316 4125 4128 3800 669 6575 4819 5977 900 1635 25 1460 619 7044 4921 648 4407 3800 1241 600 3750 5470 2313 641 4333 7539 803 316 4125 648 3530 6637 569 1985 3000 4659 5610 6917 3750 3618 1985 6887 7010 3870 900 3915 4939 7010 3870 5598 1985 1394 3397 5598 900 1635 1460 619 5708 1335 6518 4148 3750 2410 1219 6654 2252 1702 5598 803 4646 2109 6905 5520 1635 2663 885 5491 1465 4822 1722 5011 2376 4149 1903 2662 3750 803 316 2828 1767 5915 6065 2042 1335 5598 3750 2688 5598 3231 5780 7399 3750 4811 5788 1292 1641 1667 1099 4811 5393 6407 5708 6631 1335 6666 900 316 4125 4811 648 4939 6678 3750 2021 1726 340 4469 4842 4128 669 5393 4801 3154 3750 5780 7399 669 3915 544 62 5602 1913 5598 3750 3859 6759 4939 4646 1913 900 1635 1767 5915 6065 4464 5814 648 2410 1219 6654 1815 1699 6038 4231 5698 13

In [16]:
import fasttext
model = fasttext.train_supervised("train.csv", lr=1.0, wordNgrams=2, verbose=2, minCount=1, 
                                 epoch=25, loss="hs")

[***fasttext官方文档***](https://fasttext.cc/docs/en/python-module.html#train_supervised-parameters)

```python
train_supervised 参数
    input             # training file path (required)
    lr                # learning rate [0.1]
    dim               # size of word vectors [100]
    ws                # size of the context window [5]
    epoch             # number of epochs [5]
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0]
    maxn              # max length of char ngram [0]
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1]
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []
```



In [ ]:
val_pred = [model.predict(x)[0][0].split("__")[-1] for x in train_df.iloc[-5000:]["text"]]